In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
from modules.my_pyspark import *
from modules.my_drawer import MyDrawer
from pyspark.sql.types import IntegerType
from pyspark.sql.functions import avg
import pyspark.sql.functions as F

In [3]:
drawer = MyDrawer()
spark = MyPySpark(session=True, sql=True)

In [4]:
spark.context

<SparkContext master=local appName=pyspark-shell>

#### Task 1

In [5]:
file_path = r'data/flights_small.csv'

In [6]:
data = spark.readFile(file_path, 'csv')

#### Task 2

_In schema_

In [7]:
data.printSchema()

root
 |-- year: integer (nullable = true)
 |-- month: integer (nullable = true)
 |-- day: integer (nullable = true)
 |-- dep_time: string (nullable = true)
 |-- dep_delay: string (nullable = true)
 |-- arr_time: string (nullable = true)
 |-- arr_delay: string (nullable = true)
 |-- carrier: string (nullable = true)
 |-- tailnum: string (nullable = true)
 |-- flight: integer (nullable = true)
 |-- origin: string (nullable = true)
 |-- dest: string (nullable = true)
 |-- air_time: string (nullable = true)
 |-- distance: integer (nullable = true)
 |-- hour: string (nullable = true)
 |-- minute: string (nullable = true)



_Hiển thị 3 dòng đầu tiên_

In [8]:
data.toPandas().iloc[:3, :]

,year,month,day,dep_time,dep_delay,arr_time,arr_delay,carrier,tailnum,flight,origin,dest,air_time,distance,hour,minute
0,2014,12,8,658,-7,935,-5,VX,N846VA,1780,SEA,LAX,132,954,6,58
1,2014,1,22,1040,5,1505,5,AS,N559AS,851,SEA,HNL,360,2677,10,40
2,2014,3,9,1443,-2,1652,2,VX,N847VA,755,SEA,SFO,111,679,14,43


_Cho biết dữ liệu có bao nhiêu dòng_

In [9]:
data.count()

10000

#### Task 3

In [10]:
db_name = 'flights_small'

In [11]:
data.createOrReplaceTempView(db_name)

#### Task 4

_Tạo dataframe_

In [12]:
flights = spark.dataframe(db_name)

_Hiển thị 3 dòng đầu_

In [13]:
flights.toPandas().iloc[:3, :]

,year,month,day,dep_time,dep_delay,arr_time,arr_delay,carrier,tailnum,flight,origin,dest,air_time,distance,hour,minute
0,2014,12,8,658,-7,935,-5,VX,N846VA,1780,SEA,LAX,132,954,6,58
1,2014,1,22,1040,5,1505,5,AS,N559AS,851,SEA,HNL,360,2677,10,40
2,2014,3,9,1443,-2,1652,2,VX,N847VA,755,SEA,SFO,111,679,14,43


#### Task 5

In [14]:
flights = flights.withColumn('duration_hrs', flights['air_time']/60.0)

In [15]:
flights.toPandas().iloc[:3, :]

,year,month,day,dep_time,dep_delay,arr_time,arr_delay,carrier,tailnum,flight,origin,dest,air_time,distance,hour,minute,duration_hrs
0,2014,12,8,658,-7,935,-5,VX,N846VA,1780,SEA,LAX,132,954,6,58,2.20
1,2014,1,22,1040,5,1505,5,AS,N559AS,851,SEA,HNL,360,2677,10,40,6.00
2,2014,3,9,1443,-2,1652,2,VX,N847VA,755,SEA,SFO,111,679,14,43,1.85


#### Task 6

_Với dataframe_

In [16]:
long_flights1 = flights.filter(flights['distance'] > 2_000)

> Cách khác:
> ```python
> long_flights1 = flights.filter('distance > 2000')
> ```

_Số dòng thỏa điều kiện này_

In [17]:
long_flights1.count()

1481

<hr>

_Với SQL query_

In [18]:
query_0 = '''
    select fs.*
    from flights_small as fs
    where fs.distance > 2000
'''

In [19]:
long_flights1_ = spark.sqlQuery(query_0)

In [20]:
long_flights1_.count()

1481

#### Task 7

_Với dataframe_

In [21]:
time_flights = flights.filter((flights['air_time'] >= 300) & (flights['air_time'] <= 600))

> Cách khác:
> ```python
> time_flights = flights.filter('air_time >= 300 and air_time <= 600')
> ```

_Cho biết dữ liệu có bao nhiêu dòng_

In [22]:
time_flights.count()

440

<hr>

_Dùng SQL query_

In [23]:
query_1 = '''
    select fs.*
    from flights_small fs
    where fs.air_time between 300 and 600
'''

In [24]:
time_flights_ = spark.sqlQuery(query_1)

_Đếm có bao nhiêu dòng thoả điều kiện này_

In [25]:
time_flights_.count()

440

#### Task 8

In [26]:
selected1 = flights.select('origin', 'dest', 'carrier')

In [27]:
filterA = flights['origin'] == "SEA"

In [28]:
filterB = flights['dest'] == "PDX"

In [29]:
selected2 = selected1.filter(filterA & filterB)

In [30]:
selected2.show(5)

+------+----+-------+
|origin|dest|carrier|
+------+----+-------+
|   SEA| PDX|     OO|
|   SEA| PDX|     OO|
|   SEA| PDX|     OO|
|   SEA| PDX|     OO|
|   SEA| PDX|     OO|
+------+----+-------+
only showing top 5 rows



_Số dòng thoả điều kiện trên là_

In [31]:
selected2.count()

157

#### Task 9

In [32]:
avg_speed = (flights['distance'] / (flights['air_time'] / 60.0)).alias('avg_speed')

In [33]:
speed1 = flights.select('origin', 'dest', 'tailnum', avg_speed)

In [34]:
speed1.show(3)

+------+----+-------+------------------+
|origin|dest|tailnum|         avg_speed|
+------+----+-------+------------------+
|   SEA| LAX| N846VA| 433.6363636363636|
|   SEA| HNL| N559AS| 446.1666666666667|
|   SEA| SFO| N847VA|367.02702702702703|
+------+----+-------+------------------+
only showing top 3 rows



#### Task 10

In [35]:
speed2 = flights.selectExpr('origin', 'dest', 'tailnum', 'distance / (air_time)/60.0 as avg_speed')

In [36]:
speed2.show(5)

+------+----+-------+-------------------+
|origin|dest|tailnum|          avg_speed|
+------+----+-------+-------------------+
|   SEA| LAX| N846VA|0.12045454545454545|
|   SEA| HNL| N559AS|0.12393518518518518|
|   SEA| SFO| N847VA|0.10195195195195196|
|   PDX| SJC| N360SW| 0.1142570281124498|
|   SEA| BUR| N612AS|0.12296587926509187|
+------+----+-------+-------------------+
only showing top 5 rows



#### Task 11

In [37]:
flights = flights.withColumn('air_time', flights['air_time'].cast(IntegerType()))

In [38]:
flights.filter(flights['origin'] == 'PDX').groupBy().min('air_time').show()

+-------------+
|min(air_time)|
+-------------+
|           24|
+-------------+



In [39]:
flights.filter(flights['origin'] == 'SEA').groupBy().max('distance').show()

+-------------+
|max(distance)|
+-------------+
|         2724|
+-------------+



In [40]:
flights.withColumn('duration_hrs', flights['air_time'] / 60.0).groupBy().sum('duration_hrs').show()

+------------------+
| sum(duration_hrs)|
+------------------+
|25289.600000000126|
+------------------+



#### Task 12

_Nhóm theo `tailnum` và đếm_

In [41]:
by_plane = flights.groupBy('tailnum')

In [42]:
by_plane.count().show()

+-------+-----+
|tailnum|count|
+-------+-----+
| N442AS|   38|
| N102UW|    2|
| N36472|    4|
| N38451|    4|
| N73283|    4|
| N513UA|    2|
| N954WN|    5|
| N388DA|    3|
| N567AA|    1|
| N516UA|    2|
| N927DN|    1|
| N8322X|    1|
| N466SW|    1|
|  N6700|    1|
| N607AS|   45|
| N622SW|    4|
| N584AS|   31|
| N914WN|    4|
| N654AW|    2|
| N336NW|    1|
+-------+-----+
only showing top 20 rows



_Nhóm theo `origin` và đếm_

In [43]:
by_origin = flights.groupBy('origin')

In [44]:
by_origin.count().show()

+------+-----+
|origin|count|
+------+-----+
|   SEA| 6754|
|   PDX| 3246|
+------+-----+



_Nhóm theo `origin` và tính trug bình trên `air_time`_

In [45]:
flights.groupBy('origin').avg('air_time').show()

+------+------------------+
|origin|     avg(air_time)|
+------+------------------+
|   SEA| 160.4361496051259|
|   PDX|137.11543248288737|
+------+------------------+



#### Task 13

In [46]:
flights = flights.withColumn('dep_delay', flights['dep_delay'].cast(IntegerType()))

In [47]:
by_month_dest = flights.groupBy('month', 'dest')

_Tính trung bình_

In [48]:
by_month_dest.avg('dep_delay').show()

+-----+----+-------------------+
|month|dest|     avg(dep_delay)|
+-----+----+-------------------+
|    4| PHX| 1.6833333333333333|
|    1| RDM|             -1.625|
|    5| ONT| 3.5555555555555554|
|    7| OMA|               -6.5|
|    8| MDW|               7.45|
|    6| DEN|  5.418181818181818|
|    5| IAD|               -4.0|
|   12| COS|               -1.0|
|   11| ANC|  7.529411764705882|
|    5| AUS|              -0.75|
|    5| COS| 11.666666666666666|
|    2| PSP|                0.6|
|    4| ORD|0.14285714285714285|
|   10| DFW| 18.176470588235293|
|   10| DCA|               -1.5|
|    8| JNU|             18.125|
|   11| KOA|               -1.0|
|   10| OMA|-0.6666666666666666|
|    6| ONT|              9.625|
|    3| MSP|                3.2|
+-----+----+-------------------+
only showing top 20 rows



_Tính std_

In [49]:
by_month_dest.agg(F.stddev('dep_delay')).show()

+-----+----+----------------------+
|month|dest|stddev_samp(dep_delay)|
+-----+----+----------------------+
|    4| PHX|    15.003380033491737|
|    1| RDM|     8.830749846821778|
|    5| ONT|    18.895178691342874|
|    7| OMA|    2.1213203435596424|
|    8| MDW|    14.467659032985843|
|    6| DEN|    13.536905534420026|
|    5| IAD|    3.8078865529319543|
|   12| COS|    1.4142135623730951|
|   11| ANC|    18.604716401245316|
|    5| AUS|     4.031128874149275|
|    5| COS|     33.38163167571851|
|    2| PSP|     4.878524367060187|
|    4| ORD|    11.593882803741764|
|   10| DFW|     45.53019017606675|
|   10| DCA|    0.7071067811865476|
|    8| JNU|     40.79368823727514|
|   11| KOA|    1.8708286933869707|
|   10| OMA|    5.8594652770823155|
|    6| ONT|     25.98316762829351|
|    3| MSP|    21.556779370817555|
+-----+----+----------------------+
only showing top 20 rows



#### Task 14

In [50]:
file_path1 = 'data/airports.csv'

_Đọc dữ liệu_

In [51]:
airports = spark.readFile(file_path1)

_In Schema_

In [52]:
airports.printSchema()

root
 |-- faa: string (nullable = true)
 |-- name: string (nullable = true)
 |-- lat: double (nullable = true)
 |-- lon: double (nullable = true)
 |-- alt: integer (nullable = true)
 |-- tz: integer (nullable = true)
 |-- dst: string (nullable = true)



_Hiển thị 3 dòng đầu tiên_

In [53]:
airports.toPandas().iloc[:3, :]

,faa,name,lat,lon,alt,tz,dst
0,04G,Lansdowne Airport,41.130472,-80.619583,1044,-5,A
1,06A,Moton Field Municipal Airport,32.460572,-85.680028,264,-5,A
2,06C,Schaumburg Regional,41.989341,-88.101243,801,-6,A


_Cho biết dữ liệu có bao nhiêu dòng_

In [54]:
airports.count()

1397

#### Task 15

In [55]:
airports = airports.withColumnRenamed('faa', 'dest')

In [56]:
airports.show(3)

+----+--------------------+----------+-----------+----+---+---+
|dest|                name|       lat|        lon| alt| tz|dst|
+----+--------------------+----------+-----------+----+---+---+
| 04G|   Lansdowne Airport|41.1304722|-80.6195833|1044| -5|  A|
| 06A|Moton Field Munic...|32.4605722|-85.6800278| 264| -5|  A|
| 06C| Schaumburg Regional|41.9893408|-88.1012428| 801| -6|  A|
+----+--------------------+----------+-----------+----+---+---+
only showing top 3 rows



#### Task 16

In [57]:
flights_with_airports = flights.join(airports, on='dest', how='leftouter')

In [58]:
flights_with_airports.toPandas().head()

,dest,year,month,day,dep_time,dep_delay,arr_time,arr_delay,carrier,tailnum,...,distance,hour,minute,duration_hrs,name,lat,lon,alt,tz,dst
0,LAX,2014,12,8,658,-7.0,935,-5,VX,N846VA,...,954,6,58,2.200000,Los Angeles Intl,33.942536,-118.408075,126,-8,A
1,HNL,2014,1,22,1040,5.0,1505,5,AS,N559AS,...,2677,10,40,6.000000,Honolulu Intl,21.318681,-157.922428,13,-10,N
2,SFO,2014,3,9,1443,-2.0,1652,2,VX,N847VA,...,679,14,43,1.850000,San Francisco Intl,37.618972,-122.374889,13,-8,A
3,SJC,2014,4,9,1705,45.0,1839,34,WN,N360SW,...,569,17,5,1.383333,Norman Y Mineta San Jose Intl,37.362600,-121.929022,62,-8,A
4,BUR,2014,3,9,754,-1.0,1015,1,AS,N612AS,...,937,7,54,2.116667,Bob Hope,34.200667,-118.358667,778,-8,A
